# #1. Data Preprocessing 
* 분석에 사용할 데이터를 전처리하는 과정입니다. 

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd 
import dataload
import datetime as datetime
pd.set_option("max_rows", 500)
pd.set_option("max_columns", 500)
pd.set_option('float_format', '{:f}'.format)

import os 
import datetime
from tqdm.notebook import tqdm 
from functools import reduce
%matplotlib inline
tqdm.pandas()

# 1. 데이터 로드

## 1-1. 내부 데이터

In [ ]:
buy_full = dataload.load_buy()
sns_full = dataload.load_sns()

## 1-2. 기상청 데이터

In [ ]:
def load_file(weather) : 
    
    '''
    weather : 추출하고 싶은 날씨 데이터 변수 
    '''
    
    # folder path 설정 
    file_path = os.path.join('기상청데이터')
    file_list = os.listdir(file_path)
    
    # startswith('i') : i로 시작하는 파일 모두 불러오기 / endswith('csv') : csv 파일 모두 불러오기 
    data_file = sorted([file for file in file_list if file.startswith(weather)])
 
    # csv 파일들을 DataFrame으로 불러와서 concat
    df = pd.DataFrame()
    for d in data_file:
        data = pd.read_csv(os.path.join(file_path, d))
        df = pd.concat([df, data], axis=0).reset_index(drop=True)
     
    df = df.iloc[:,1:]
    
    return df

In [ ]:
# 지역명 부여 함수
def make_region(aws_id):
    region = {'서울':108, '부산':159, '대구':143, '인천':112, '광주':156, '대전':133, '울산':152, '수원':119, 
              '강릉':105, '청주':131, '천안':232, '전주':146, '여수':168, '안동':136, '창원':155, '제주':184}
    res = None
    for r, code in region.items():
        if aws_id == code:
             res = r
    return res

# 해당 지역, 해당 년월의 평균치로 결측치를 대체하는 함수

def missing_value(data, aws_id, year, month, var):
    data['year'] = data['date'].apply(lambda x : x.year)
    data['month'] = data['date'].apply(lambda x : x.month)
    sample = data[(data.aws_id==aws_id)&(data.year==year)&(data.month==month)]
    alt = sample[var].mean() # 평균
    # 대체
    data[(data.aws_id==aws_id)&(data.year==year)&(data.month==month)] = data[(data.aws_id==aws_id)&(data.year==year)&(data.month==month)].fillna(alt) 
    data = data.drop(['year', 'month'], axis=1) # 월 변수 제거
    return data

### 1-2-1. 습도

In [ ]:
tqdm.pandas()

humid = load_file('humid')
humid['date'] = humid['tm'].progress_apply(lambda x : pd.to_datetime(x[:10]))
humid2 = humid.groupby(['date', 'aws_id']).max().reset_index()[['date', 'aws_id', 'hm_max']]

humid2['region'] = humid2['aws_id'].apply(make_region)
humid2 = missing_value(humid2, 232, 2019, 3, 'hm_max') # 충남(천안)
humid2 = missing_value(humid2, 232, 2019, 4, 'hm_max') # 대구
humid2 = missing_value(humid2, 143, 2019, 4, 'hm_max') # 대구

  0%|          | 0/280320 [00:00<?, ?it/s]

In [ ]:
humid2.head()

,date,aws_id,hm_max,region
0,2018-01-01,105,25.400000,강릉
1,2018-01-01,108,57.100000,서울
2,2018-01-01,112,67.200000,인천
3,2018-01-01,119,84.700000,수원
4,2018-01-01,131,71.400000,청주


In [ ]:
humid2.isnull().sum()

date      0
aws_id    0
hm_max    0
region    0
dtype: int64

### 1-2-2. 일조시간

In [ ]:
tqdm.pandas()

sun = load_file('sun')
sun['date'] = sun['tma'].progress_apply(lambda x : pd.to_datetime(x[:10]))
sun2 = sun.groupby(['date', 'stn_id']).max().reset_index()[['date', 'stn_id', 'sum_ss_hr']]
sun2.columns = ['date', 'aws_id', 'sum_ss_hr']

# 지역명 부여
sun2['region'] = sun2['aws_id'].apply(make_region)

  0%|          | 0/11674 [00:00<?, ?it/s]

In [ ]:
sun2.head()

,date,aws_id,sum_ss_hr,region
0,2018-01-01,105,57.900000,강릉
1,2018-01-01,108,51.600000,서울
2,2018-01-01,112,53.800000,인천
3,2018-01-01,119,52.700000,수원
4,2018-01-01,131,54.600000,청주


In [ ]:
sun2.isnull().sum()

date         0
aws_id       0
sum_ss_hr    0
region       0
dtype: int64

### 1-2-3. 기압

In [ ]:
tqdm.pandas()

press = load_file('press')
press['date'] = press['tma'].progress_apply(lambda x : pd.to_datetime(x[:10]))
press2 = press.groupby(['date', 'stn_id']).max().reset_index()[['date', 'stn_id', 'max_pa']]
press2.columns = ['date', 'aws_id', 'max_pa']

# 지역명 부여
press2['region'] = press2['aws_id'].apply(make_region)

# 결측치 처리
press2 = missing_value(press2, 143, 2019, 4, 'max_pa') # 대구
press2 = missing_value(press2, 105, 2019, 12, 'max_pa') # 강릉
press2 = missing_value(press2, 105, 2020, 1, 'max_pa') # 강릉

  0%|          | 0/11680 [00:00<?, ?it/s]

In [ ]:
press2.head()

,date,aws_id,max_pa,region
0,2018-01-01,105,1023.000000,강릉
1,2018-01-01,108,1018.100000,서울
2,2018-01-01,112,1020.300000,인천
3,2018-01-01,119,1025.100000,수원
4,2018-01-01,131,1022.000000,청주


In [ ]:
press2.isnull().sum()

date      0
aws_id    0
max_pa    0
region    0
dtype: int64

### 1-2-4. 기온  
* 일별 데이터 : 시각, 지점번호, 평균기온, 최고기온, 최저기온

In [ ]:
tqdm.pandas()
temp['date'] = temp['tma'].progress_apply(lambda x : pd.to_datetime(x[:10]))

100%|██████████| 17536/17536 [00:01<00:00, 14506.81it/s]


In [ ]:
temp[temp["avg_ta"].isna()]

,tma,stn_id,avg_ta,max_ta,min_ta,date
8449,2019-11-25 00:00:00.0,232,nan,6.800000,0.500000,2019-11-25
8458,2019-11-26 00:00:00.0,232,nan,13.400000,nan,2019-11-26


In [ ]:
# 실제 기상청에도 자료가 없어서, avg_ta = (max+min)/2 로 결측값 채움 
temp.loc[8449, 'avg_ta'] = (temp.loc[8449]['max_ta'] + temp.loc[8449]['min_ta']) / 2 

temp.loc[8458, 'min_ta'] = 8.2
temp.loc[8458, 'avg_ta'] = (temp.loc[8458]['max_ta'] + temp.loc[8458]['min_ta']) / 2 

In [ ]:
temp.isnull().sum()

tma       0
stn_id    0
avg_ta    0
max_ta    0
min_ta    0
date      0
dtype: int64

### 1-2-5. 강수량  
* 관측시간, AWS번호, RN_DAY (누적 강수량, 마지막 23시 데이터), RN_HR1 (1시간 강수량, 일별로 최대값)

In [ ]:
rain = load_file('rain')
rain['date'] = rain['tm'].progress_apply(lambda x : pd.to_datetime(x[:10]))

100%|██████████| 420864/420864 [00:27<00:00, 15085.17it/s]


In [ ]:
print("관측 지역 수 : ", rain.aws_id.nunique())
print(rain.aws_id.unique())

관측 지역 수 :  16
[133 119 136 155 105 108 156 184 159 112 232 131 143 146 152 168]


In [ ]:
rain = rain.sort_values('tm').groupby(['date', 'aws_id']).agg({
        'rn_day' : [('rn_day', 'last')], 
        'rn_hr1' : [('rn_hr1', np.max)]
}).reset_index()
rain.columns = rain.columns.get_level_values(level=0)

In [ ]:
'''
rn_day : 일별 강수량 
rn_hr1 : 일별 1시간 최다 강수량 
'''

print(rain.shape)
rain.head().append(rain.tail())

(17536, 4)


,date,aws_id,rn_day,rn_hr1
0,2018-01-01,105,0.000000,0.000000
1,2018-01-01,108,0.000000,0.000000
2,2018-01-01,112,0.000000,0.000000
3,2018-01-01,119,0.000000,0.000000
4,2018-01-01,131,0.000000,0.000000
17531,2020-12-31,156,0.600000,0.300000
17532,2020-12-31,159,0.000000,0.000000
17533,2020-12-31,168,0.000000,0.000000
17534,2020-12-31,184,3.800000,2.400000
17535,2020-12-31,232,0.000000,0.000000


In [ ]:
# 결측값 확인 
rain.isnull().sum()

date       0
aws_id     0
rn_day    16
rn_hr1    17
dtype: int64

In [ ]:
# 결측값 0 으로 채우기
rain = rain.fillna(0)

In [ ]:
rain.isnull().sum()

date      0
aws_id    0
rn_day    0
rn_hr1    0
dtype: int64

### 1-2-6. 풍속  
* 관측시간, 지점번호, 평균풍속

In [ ]:
wind = load_file('wind')
wind['date'] = wind['tma'].progress_apply(lambda x : pd.to_datetime(x[:10]))

100%|██████████| 17536/17536 [00:01<00:00, 15200.28it/s]


In [ ]:
'''
avg_ws : 평균풍속 
'''
wind.head().append(wind.tail())

,tma,stn_id,avg_ws,date
0,2018-01-01 00:00:00.0,112,1.600000,2018-01-01
1,2018-01-01 00:00:00.0,155,1.800000,2018-01-01
2,2018-01-01 00:00:00.0,168,4.500000,2018-01-01
3,2018-01-01 00:00:00.0,184,3.500000,2018-01-01
4,2018-01-02 00:00:00.0,119,1.600000,2018-01-02
17531,2020-12-30 00:00:00.0,232,3.300000,2020-12-30
17532,2020-12-31 00:00:00.0,112,3.000000,2020-12-31
17533,2020-12-31 00:00:00.0,152,3.300000,2020-12-31
17534,2020-12-31 00:00:00.0,155,2.000000,2020-12-31
17535,2020-12-31 00:00:00.0,159,4.100000,2020-12-31


In [ ]:
print("관측 지역 수 : ", wind.stn_id.nunique())
print(wind.stn_id.unique())

관측 지역 수 :  16
[112 155 168 184 119 136 156 105 131 133 143 146 159 232 108 152]


In [ ]:
wind.isnull().sum()

tma        0
stn_id     0
avg_ws    11
date       0
dtype: int64

In [ ]:
# 결측치 제거
wind = missing_value(wind, 112, 2018, 7, 'avg_ws')
wind = missing_value(wind, 143, 2019, 4, 'avg_ws')
wind = missing_value(wind, 133, 2019, 5, 'avg_ws')
wind = missing_value(wind, 152, 2019, 6, 'avg_ws')
wind = missing_value(wind, 131, 2019, 7, 'avg_ws')
wind = missing_value(wind, 232, 2019, 11, 'avg_ws')
wind = missing_value(wind, 133, 2019, 11, 'avg_ws')
# wind = missing_value(wind, 159, 2020, 3, 'avg_ws')

In [ ]:
wind.isnull().sum()

tma       0
stn_id    0
avg_ws    0
date      0
dtype: int64

## 1-3. 외부 데이터

### 1-3-1. 미세먼지

In [ ]:
def load_dust(year) : 
    
    # folder path 설정 
    file_path = os.path.join('외부데이터', year)
    file_list = os.listdir(file_path)
    
    # 파일 불러오기
    data_file = sorted([file for file in file_list if file.startswith(year)])
    
    # csv 파일들을 DataFrame으로 불러와서 concat
    df = pd.DataFrame()
    for d in data_file:
        data = pd.read_excel(os.path.join(file_path, d), engine='openpyxl')
        df = pd.concat([df, data], axis=0).reset_index(drop=True)
    df = df[~df.지역.isna()] # 지역값이 없는 경우 제거
    df = df[~df.PM10.isna()]
    df = df[~df.PM25.isna()]
    
    # 지역 리스트
    region = {'서울':108, '부산':159, '대구':143, '인천':112, '광주':156, '대전':133, '울산':152, '수원':119, 
              '강릉':105, '청주':131, '천안':232, '전주':146, '여수':168, '안동':136, '창원':155, '제주':184}
    
    # 시간별 데이터를 일별 데이터로 변환
    # 지역별 평균값
    data = pd.DataFrame()
    for r, code in region.items(): # r:지명, code:지역코드
        sample = df.copy()
        sample['date'] = sample['측정일시'].apply(lambda x : str(x)[:8])
        sample['region'] = sample['지역'].apply(lambda x : r if r in x else None)
        sample = sample[sample.region == r]
        sample['aws_id'] = code
        sample = sample.groupby(['date', 'region', 'aws_id']).mean().reset_index()[['date', 'region', 'aws_id', 'PM10', 'PM25']]
        data = pd.concat([data, sample], axis=0).reset_index(drop=True)

    return data

In [ ]:
dust2018 = load_dust('2018')
dust2019 = load_dust('2019')

In [ ]:
dust = pd.concat([dust2018, dust2019], axis=0).reset_index(drop=True)
dust.shape

(11485, 5)

In [ ]:
dust.columns = ['date', 'region', 'aws_id', 'PM10', 'PM25']
dust['date'] = dust['date'].progress_apply(lambda x : pd.to_datetime(x))

  0%|          | 0/11485 [00:00<?, ?it/s]

In [ ]:
dust.head()

,date,region,aws_id,PM10,PM25
0,2018-01-01,서울,108,42.307692,21.470696
1,2018-01-02,서울,108,40.470588,22.711397
2,2018-01-03,서울,108,35.242478,19.042478
3,2018-01-04,서울,108,46.385027,25.381462
4,2018-01-05,서울,108,57.841918,37.159858


### 1-3-2. 검색어 트렌드 크롤링  
* https://datalab.naver.com/keyword/trendSearch.naver 
* **sm_cat** 에 해당하는 검색어 트렌드 값 산출 
* `/` 로 나뉘어 있는 경우, 평균값 사용

In [ ]:
# Data Load & Get Category
buy_full = dataload.load_buy()
print(buy_full.shape)

food = list(buy_full[buy_full["big_cat"]=="식품"].sm_cat.unique())
beauty = list(buy_full[buy_full["big_cat"]=="뷰티"].sm_cat.unique())
app = list(buy_full[buy_full["big_cat"]=="냉난방가전"].sm_cat.unique())
print("food : {} | beauty : {} | app : {}".format(len(food), len(beauty), len(app)))

(2056899, 6)
food : 212 | beauty : 131 | app : 40


In [ ]:
# original category 
pd.DataFrame(food).to_csv(os.path.join("최종데이터", "food_category.csv"), encoding="cp949")
pd.DataFrame(beauty).to_csv(os.path.join("최종데이터", "beauty_category.csv"), encoding="cp949")
pd.DataFrame(app).to_csv(os.path.join("최종데이터", "app_category.csv"), encoding="cp949")

In [ ]:
# category 수정 
food2 = pd.read_csv(os.path.join("최종데이터", "food_category.csv"), encoding="cp949")
beauty2 = pd.read_csv(os.path.join("최종데이터", "beauty_category.csv"), encoding="cp949")
app2 = pd.read_csv(os.path.join("최종데이터", "app_category.csv"), encoding="cp949")

In [ ]:
food2 = food2['0'].tolist()
beauty2 = beauty2['0'].tolist()
app2 = app2['0'].tolist()

print("food : {} | beauty : {} | app : {}".format(len(food2), len(beauty2), len(app2)))

food : 212 | beauty : 131 | app : 40


In [ ]:
# original category 정보 저장 
food_cat = pd.concat([pd.DataFrame(food), pd.DataFrame(food2)], axis=1)
beauty_cat = pd.concat([pd.DataFrame(beauty), pd.DataFrame(beauty2)], axis=1)
app_cat = pd.concat([pd.DataFrame(app), pd.DataFrame(app2)], axis=1)

#### 크롤링

In [ ]:
client_id = "???" ; client_secret = "???"

In [ ]:
def keyword_search(client_id, client_secret, keyword) : 
    ages = {20:['3','4'], 30:['5','6'], 40:['7','8'], 50:['9','10'], 60:['11']} ; genders = ["f", "m"]
    df = pd.DataFrame()
    
    for age, age_list in ages.items() : 
        for gender in genders : 
            client_id = client_id ; client_secret = client_secret
            url = "https://openapi.naver.com/v1/datalab/search"
            body = {"startDate":"2018-01-01",
                    "endDate":"2019-12-31",
                    "timeUnit":"date",
                    "keywordGroups":[{"groupName":keyword,"keywords":[keyword]}],
                    "ages": age_list ,
                    "gender": gender}
            body_j = json.dumps(body)

            # get results 
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            request.add_header("Content-Type","application/json")
            response = urllib.request.urlopen(request, data=body_j.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                scraped = response_body.decode('utf-8')
                
                try : 
                    result = json_normalize(json.loads(scraped)["results"][0]["data"])
                    result.insert(1, "age", age) ; result.insert(1, "sex", gender.upper()) 
                    df = df.append(result, ignore_index=True)
                except : 
                    pass

            else:
                print("Error Code:" + rescode) 
    
    try : 
        df.insert(3, "sm_cat", keyword)
    except : 
        pass 
    
    return df

In [ ]:
def category_search(big_cat) : 
    df = pd.DataFrame()
    
    for sm_cat in tqdm(big_cat) : 
        try: 
            if not '/' in sm_cat : 
                tmp = keyword_search(client_id, client_secret, sm_cat)
                df = df.append(tmp, ignore_index=True)

            # '/' 나뉘어져 있는 경우 
            else : 
                tmp2 = pd.DataFrame()
                sm_cats = sm_cat.split('/')
                for sm in sm_cats : 
                    tmp = keyword_search(client_id, client_secret, sm)
                    tmp2 = tmp2.append(tmp, ignore_index=True)
                    
                try : 
                    tmp2 = tmp2.groupby(["period", "sex", "age"]).agg({'ratio' : [('ratio', np.mean)]}).reset_index()
                    tmp2.columns = tmp2.columns.get_level_values(0)
                    tmp2.insert(3, "sm_cat", sm_cat)
                except : 
                    pass

                df = df.append(tmp2, ignore_index=True)
        
        except urllib.request.HTTPError : 
            print("Too Many Requests - at {}".format(big_cat.index(sm_cat)))
            
    if big_cat[0] in food2 : df.insert(3, "big_cat", "식품")
    elif big_cat[0] in beauty2 : df.insert(3, "big_cat", "뷰티")
    elif big_cat[0] in app2 : df.insert(3, "big_cat", "냉난방가전")
    else : df.insert(3, "big_cat", "날씨")
        
    return df 

#### 날씨검색량데이터 `weather_trend.csv`

In [ ]:
weather_keywords = ["오늘날씨", "미세먼지", "오늘 비", "기상정보", "태풍정보"]
weather_search = category_search(weather_keywords)

100%|██████████| 5/5 [00:21<00:00,  4.28s/it]


In [ ]:
print(weather_search.shape)
weather_search.head(10)

(33070, 6)


,period,sex,age,big_cat,sm_cat,ratio
0,2018-01-01,F,20,날씨,오늘날씨,10.69387
1,2018-01-02,F,20,날씨,오늘날씨,16.83434
2,2018-01-03,F,20,날씨,오늘날씨,13.63583
3,2018-01-04,F,20,날씨,오늘날씨,12.33925
4,2018-01-05,F,20,날씨,오늘날씨,12.84071
5,2018-01-06,F,20,날씨,오늘날씨,12.89477
6,2018-01-07,F,20,날씨,오늘날씨,19.33421
7,2018-01-08,F,20,날씨,오늘날씨,34.82321
8,2018-01-09,F,20,날씨,오늘날씨,22.80943
9,2018-01-10,F,20,날씨,오늘날씨,16.20355


In [ ]:
weather_search.sm_cat.value_counts()

미세먼지    7300
오늘날씨    7300
기상정보    6928
태풍정보    6143
오늘 비    5399
Name: sm_cat, dtype: int64

In [ ]:
weather_search.to_csv("최종데이터/weather_trend.csv", index=False, encoding='cp949')

### 1-3-3. 소비자물가지수

In [ ]:
df_seoul = pd.read_csv('외부데이터/소비자심리지수_seoul_past.csv', encoding='CP949')
df_others = pd.read_csv('외부데이터/소비자심리지수_other_past.csv', encoding='CP949')

In [ ]:
df_seoul.head(2)

,지수코드별,분류코드별,항목,단위,2018. 01 월,2018. 02 월,2018. 03 월,2018. 04 월,2018. 05 월,2018. 06 월,2018. 07 월,2018. 08 월,2018. 09 월,2018. 10 월,2018. 11 월,2018. 12 월,2019. 01 월,2019. 02 월,2019. 03 월,2019. 04 월,2019. 05 월,2019. 06 월,2019. 07 월,2019. 08 월,2019. 09 월,2019. 10 월,2019. 11 월,2019. 12 월,Unnamed: 28
0,현재생활형편CSI,전체,소비자동향,NaN,94.0,94,95.0,95.0,95.0,94.0,91.0,89.0,90.0,91.0,90.0,89.0,90.0,93.0,91.0,93.0,91.0,91.0,91.0,90.0,92.0,92.0,92.0,92.0,NaN
1,현재생활형편CSI,남자,소비자동향,NaN,94.0,94,95.0,94.0,95.0,94.0,92.0,89.0,90.0,91.0,90.0,90.0,90.0,93.0,91.0,92.0,91.0,91.0,90.0,89.0,92.0,91.0,92.0,91.0,NaN


# 2. 데이터 병합

## 2-1. `weather_final.csv`

### 2-1-1. `weather1819_2.csv`

In [ ]:
dust.shape, humid2.shape, sun2.shape, press2.shape # total = 11485

((11485, 5), (11680, 4), (11674, 4), (11680, 4))

In [ ]:
weather = reduce(lambda left, right : pd.merge(left, right, how='left', on=['date', 'aws_id', 'region']), [humid2, dust, sun2, press2])
weather = weather[['date', 'aws_id', 'region', 'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa']]

In [ ]:
print(weather.shape)
weather.head()

(11680, 8)


,date,aws_id,region,PM10,PM25,hm_max,sum_ss_hr,max_pa
0,2018-01-01,105,강릉,20.066667,13.400000,25.400000,57.900000,1023.000000
1,2018-01-01,108,서울,42.307692,21.470696,57.100000,51.600000,1018.100000
2,2018-01-01,112,인천,37.518681,18.641758,67.200000,53.800000,1020.300000
3,2018-01-01,119,수원,42.782895,21.375000,84.700000,52.700000,1025.100000
4,2018-01-01,131,청주,51.349650,32.706294,71.400000,54.600000,1022.000000


In [ ]:
weather.to_csv('최종데이터/weather1819_2.csv', encoding='cp949')

### 2-1-2. `weather1819.csv`

In [ ]:
rain = rain.rename(columns = {'aws_id' : 'stn_id'})

In [ ]:
weather = reduce(lambda left, right: pd.merge(left, right, on=['date', 'stn_id']), [temp.drop("tma", axis=1), rain, wind.drop("tma", axis=1)])
weather = weather.reindex(columns=['date', 'stn_id', 'avg_ta', 'max_ta', 'min_ta', 'rn_day', 'rn_hr1', 'avg_ws'])

In [ ]:
print(weather.shape)
weather.head().append(weather.tail())

(11680, 8)


,date,stn_id,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws
0,2018-01-01,105,1.300000,5.700000,-2.100000,0.000000,0.000000,3.700000
1,2018-01-01,112,-0.300000,2.700000,-2.700000,0.000000,0.000000,1.600000
2,2018-01-01,119,-1.700000,4.700000,-6.900000,0.000000,0.000000,1.000000
3,2018-01-01,136,-1.000000,4.700000,-6.500000,0.000000,0.000000,2.200000
4,2018-01-01,152,2.100000,6.200000,-0.400000,0.000000,0.000000,3.300000
11675,2019-12-31,112,-7.300000,-5.300000,-9.900000,0.000000,0.000000,5.000000
11676,2019-12-31,131,-5.700000,-1.400000,-8.000000,0.000000,0.000000,2.200000
11677,2019-12-31,143,-2.500000,2.600000,-5.100000,0.000000,0.000000,5.200000
11678,2019-12-31,155,-1.700000,4.800000,-4.000000,0.000000,0.000000,2.500000
11679,2019-12-31,184,2.900000,6.400000,1.800000,0.000000,0.000000,5.400000


In [ ]:
weather.to_csv('최종데이터/weather1819.csv', index=False)

### 2-1-3. 파생변수 생성

#### 날씨관련 파생변수

In [ ]:
# 체감온도 : 외부에 있는 사람이나 동물이 바람과 한기에 노출된 피부로 부터 열을 빼앗길 때 느끼는 추운 정도를 나타내는 지수
weather['체감온도'] = 13.12 + 0.6215*weather['평균기온'] - 11.37 * weather['평균풍속'] * 0.16 + 0.3965 * weather['평균풍속'] * 0.16 * weather['평균기온']

# 열지수 : 기온과 습도에 따라 사람이 실제로 느끼는 더위를 지수화한 것
H = weather['1시간최대습도']; T = weather['평균기온']
RH = H / sum(H) * 100 # percentage
weather['열지수'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH

# 폭염여부 : 일 최고기온이 33℃ 이상인 날
weather['폭염여부'] = weather['최고기온'].apply(lambda x : 1 if x>=33 else 0)

# 강수여부 : 일강수량이 0.1mm 이상인 날
weather['강수여부'] = weather['일별강수량'].apply(lambda x : 1 if x>=0.1 else 0)

In [ ]:
weather['폭염여부'].value_counts()

0    10979
1      701
Name: 폭염여부, dtype: int64

In [ ]:
weather['강수여부'].value_counts()

0    8766
1    2914
Name: 강수여부, dtype: int64

#### 시간관련 파생변수

In [ ]:
weather['날짜'] = weather['날짜'].apply(lambda x : pd.to_datetime(x))
weather['연']  = weather['날짜'].dt.year
weather['월']  = weather['날짜'].dt.month
weather['일']  = weather['날짜'].dt.day
weather["분기"] = weather['날짜'].dt.quarter
weather['요일']  = weather['날짜'].dt.weekday # 월 0 화 1 수 2 목 3 금 4 토 5 일 6 

In [ ]:
# 공휴일 정보 추출 함수
def getHoliday(year):
    
    '''
    공공데이터포털 공휴일 정보 OPEN API  
    '''
    
    url = f'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&ServiceKey={key}&_type=json&numOfRows=20'
    response = requests.get(url)
    holidays = response.json()['response']['body']['items']['item']
    holidays = pd.DataFrame(holidays)
    holidays['locdate'] = holidays['locdate'].astype(str).apply(lambda x : '-'.join([x[:4], x[4:6], x[6:]]))
    
    return holidays

key = '8H1yac%2Bb0yetY2Waad%2BQIRU43O17onRUd7iR0k2p6%2B4i0yNYn3ym0cXgt3ZqWrR9uFw%2BDu%2B5quHbsXxjpsw5ng%3D%3D'

# 공휴일 관련 변수
holidays2018 = getHoliday(2018)
holidays2019 = getHoliday(2019)

holidays = pd.concat([holidays2018, holidays2019], axis=0, ignore_index = True)
holidays["locdate"] = holidays['locdate'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))
holidays = holidays.drop(['dateKind', 'seq'], axis=1)
holidays.columns = ['공휴일명', '공휴일여부', '날짜']

weather = weather.merge(holidays[['날짜', '공휴일명', '공휴일여부']], on='날짜', how='left')
weather['공휴일여부'] = weather['공휴일여부'].apply(lambda x : 1 if x == 'Y' else 0)

In [ ]:
# 주말 : [5,6]
weather['주말여부'] = weather['요일'].apply(lambda x : 1 if x in [5,6] else 0)

# 계절 : 봄 0 여름 1 가을 2 겨울 3 
weather["계절"] = weather["월"].apply(lambda x : 0 if x in [3,4,5] else 1 if x in [6,7,8] 
                                   else 2 if x in [9,10,11] else 3)

### 2-1-4. 결측치 처리

In [ ]:
# 해당 지역, 해당 년월의 평균치로 결측치를 대체하는 함수

def missing_value2(data, aws_id, year, var):
    month_list = data[(data[var].isnull())&(data.지점번호==aws_id)&(data.year==year)]['month'].unique()
    for month in month_list:
        sample = data[(data.지점번호==aws_id)&(data.year==year)&(data.month==month)&(~data[var].isnull())]
        alt = sample[var].mean() # 평균
        # 대체
        if np.isnan(alt)==True: # 해당 년,월의 변수값이 하나도 없으면 전년도 월평균으로 대체
            data.loc[(data.지점번호==aws_id)&(data.year==year)&(data.month==month),var] = data.loc[(data.지점번호==aws_id)&(data.month==month), var].mean()
        else:
            data.loc[(data.지점번호==aws_id)&(data.year==year)&(data.month==month),var] = alt 
    return data

In [ ]:
#### PM10 ####
# 안동
weather = missing_value2(weather, 136, 2018, 'PM10')
weather = missing_value2(weather, 136, 2019, 'PM10')
# 제주
weather = missing_value2(weather, 184, 2018, 'PM10')
weather = missing_value2(weather, 184, 2019, 'PM10')
# 천안
weather = missing_value2(weather, 232, 2018, 'PM10')
weather = missing_value2(weather, 232, 2019, 'PM10')
# 강릉 
weather = missing_value2(weather, 105, 2018, 'PM10')
weather = missing_value2(weather, 105, 2019, 'PM10')


#### PM25 ####
# 안동
weather = missing_value2(weather, 136, 2018, 'PM25')
weather = missing_value2(weather, 136, 2019, 'PM25')
# 제주
weather = missing_value2(weather, 184, 2018, 'PM25')
weather = missing_value2(weather, 184, 2019, 'PM25')
# 천안
weather = missing_value2(weather, 232, 2018, 'PM25')
weather = missing_value2(weather, 232, 2019, 'PM25')
# 강릉 
weather = missing_value2(weather, 105, 2018, 'PM25')
weather = missing_value2(weather, 105, 2019, 'PM25')

#### 일조시간 ####
# 대구
weather = missing_value2(weather, 143, 2018, '일조시간합')
weather = missing_value2(weather, 143, 2019, '일조시간합')
# 부산 
weather = missing_value2(weather, 159, 2018, '일조시간합')
weather = missing_value2(weather, 159, 2019, '일조시간합')
# 서울
weather = missing_value2(weather, 108, 2018, '일조시간합')
weather = missing_value2(weather, 108, 2019, '일조시간합')

In [ ]:
weather.to_csv('최종데이터/weather_final.csv', encoding='cp949', index=None)

## 2-2. `weather_final2.csv`

In [ ]:
weather['연']  = weather['날짜'].dt.year
weather['월']  = weather['날짜'].dt.month
weather.to_csv('최종데이터/weather_final2.csv', encoding='cp949', index=None)

## 2-3. 검색어 트렌드 `trendsearch.csv`

In [ ]:
buy_full['date'] = buy_full['date'].progress_apply(lambda x : pd.to_datetime(str(x), format='%Y%m%d'))

100%|██████████| 2056899/2056899 [03:23<00:00, 10095.74it/s]


In [ ]:
search = pd.concat([app_search2, beauty_search, food_search], axis=0).reset_index(drop=True)
search.shape

(2253744, 6)

In [ ]:
search = search.rename(columns = {'period' : 'date'})
search['date'] = search['date'].progress_apply(lambda x : pd.to_datetime(str(x), format='%Y-%m-%d'))

100%|██████████| 2253744/2253744 [02:43<00:00, 13780.29it/s]


In [ ]:
trend = buy_full.merge(search, on=["date", "sex", "age", "big_cat", "sm_cat"], how='outer').fillna(0)
trend = trend.sort_values(['date', 'sm_cat', 'sex', 'age'], ascending=True).reset_index(drop=True)

print(trend.shape)
trend.head()

(2568668, 7)


,date,sex,age,big_cat,sm_cat,qty,ratio
0,2018-01-01,F,20,식품,가공란,37.0,0.0
1,2018-01-01,F,30,식품,가공란,16.0,0.0
2,2018-01-01,F,40,식품,가공란,9.0,0.0
3,2018-01-01,F,50,식품,가공란,3.0,0.0
4,2018-01-01,M,20,식품,가공란,13.0,0.0


In [ ]:
# Data Save 
trend.to_csv(os.path.join("외부데이터", "trendsearch.csv"), index=False, encoding='cp949')

## 2-4. 날씨 정보 `trend_with_weather.csv`

In [ ]:
weather_search = weather_search.rename(columns = {'period' : 'date'})
weather_search['date'] = weather_search['date'].progress_apply(lambda x : pd.to_datetime(str(x), format='%Y-%m-%d'))

100%|██████████| 33070/33070 [00:03<00:00, 10952.54it/s]


In [ ]:
for w_trend in weather_search.sm_cat.unique() : 
    tmp = weather_search[weather_search["sm_cat"] == w_trend][['date', 'sex', 'age', 'ratio']].rename(columns = {'ratio' : w_trend})
    trend = trend.merge(tmp, on=['date', 'sex', 'age'], how='left')

In [ ]:
trend = trend.sort_values(['date', 'sm_cat', 'sex', 'age'], ascending=True).reset_index(drop=True)
print(trend.shape)
trend.head().append(trend.tail())

(2568668, 12)


,date,sex,age,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보
0,2018-01-01,F,20,식품,가공란,37.0,0.00000,10.69387,4.09137,NaN,NaN,0.04440
1,2018-01-01,F,30,식품,가공란,16.0,0.00000,12.42969,6.35294,NaN,0.51282,0.42417
2,2018-01-01,F,40,식품,가공란,9.0,0.00000,13.02249,4.88037,NaN,0.40650,NaN
3,2018-01-01,F,50,식품,가공란,3.0,0.00000,11.69080,4.53350,1.21951,0.90909,NaN
4,2018-01-01,M,20,식품,가공란,13.0,0.00000,11.23032,3.61296,0.38910,2.22222,0.04060
2568663,2019-12-31,M,20,냉난방가전,히터,8.0,25.60296,28.46931,1.27392,1.36186,7.40740,0.08120
2568664,2019-12-31,M,30,냉난방가전,히터,22.0,24.16356,31.70578,1.41621,NaN,3.63636,NaN
2568665,2019-12-31,M,40,냉난방가전,히터,38.0,25.11627,37.03963,1.60475,NaN,4.90956,0.17814
2568666,2019-12-31,M,50,냉난방가전,히터,23.0,25.18518,49.94212,2.41013,NaN,4.81481,0.10649
2568667,2019-12-31,M,60,냉난방가전,히터,10.0,21.95121,67.48388,4.62476,NaN,19.13043,0.69605


In [ ]:
trend = trend.fillna(0)

In [ ]:
trend.head().append(trend.tail())

,date,sex,age,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보
0,2018-01-01,F,20,식품,가공란,37.0,0.00000,10.69387,4.09137,0.00000,0.00000,0.04440
1,2018-01-01,F,30,식품,가공란,16.0,0.00000,12.42969,6.35294,0.00000,0.51282,0.42417
2,2018-01-01,F,40,식품,가공란,9.0,0.00000,13.02249,4.88037,0.00000,0.40650,0.00000
3,2018-01-01,F,50,식품,가공란,3.0,0.00000,11.69080,4.53350,1.21951,0.90909,0.00000
4,2018-01-01,M,20,식품,가공란,13.0,0.00000,11.23032,3.61296,0.38910,2.22222,0.04060
2568663,2019-12-31,M,20,냉난방가전,히터,8.0,25.60296,28.46931,1.27392,1.36186,7.40740,0.08120
2568664,2019-12-31,M,30,냉난방가전,히터,22.0,24.16356,31.70578,1.41621,0.00000,3.63636,0.00000
2568665,2019-12-31,M,40,냉난방가전,히터,38.0,25.11627,37.03963,1.60475,0.00000,4.90956,0.17814
2568666,2019-12-31,M,50,냉난방가전,히터,23.0,25.18518,49.94212,2.41013,0.00000,4.81481,0.10649
2568667,2019-12-31,M,60,냉난방가전,히터,10.0,21.95121,67.48388,4.62476,0.00000,19.13043,0.69605


In [ ]:
trend.to_csv(os.path.join("외부데이터", "trend_with_weather.csv"), index=False, encoding='cp949')

## 2-5. 날씨지수 `data_with_weather.csv`

### 2-5-1. 서울 소비자 심리지수 계산

In [ ]:
df_seoul1 = df_seoul[df_seoul['분류코드별']=='서울'].reset_index(drop=True)

In [ ]:
print(df_seoul1.shape)

(24, 29)


In [ ]:
df_seoul1['지수코드별'].unique()

array(['현재생활형편CSI', '현재경기판단CSI', '생활형편전망CSI', '향후경기전망CSI', '취업기회전망CSI',
       '금리수준전망CSI', '가계수입전망CSI', '소비지출전망CSI', '내구재 지출전망CSI',
       '의류비 지출전망CSI', '외식비 지출전망CSI', '여행비 지출전망CSI', '교육비 지출전망CSI',
       '의료·보건비 지출전망CSI', '교양·오락·문화생활비 지출전망CSI', '교통비 및 통신비 지출전망CSI',
       '주거비 지출전망CSI', '현재가계저축CSI', '가계저축전망CSI', '현재가계부채CSI', '가계부채전망CSI',
       '물가수준전망(1년후)CSI', '주택가격전망CSI', '임금수준전망CSI'], dtype=object)

In [ ]:
df_csi = df_seoul1[df_seoul1['지수코드별'].apply(lambda x: x in ["현재생활형편CSI","생활형편전망CSI","현재경기판단CSI","향후경기전망CSI","소비지출전망CSI","가계수입전망CSI"])].reset_index(drop=True)

In [ ]:
df_csi.head(2)

,지수코드별,분류코드별,항목,단위,2018. 01 월,2018. 02 월,2018. 03 월,2018. 04 월,2018. 05 월,2018. 06 월,2018. 07 월,2018. 08 월,2018. 09 월,2018. 10 월,2018. 11 월,2018. 12 월,2019. 01 월,2019. 02 월,2019. 03 월,2019. 04 월,2019. 05 월,2019. 06 월,2019. 07 월,2019. 08 월,2019. 09 월,2019. 10 월,2019. 11 월,2019. 12 월,Unnamed: 28
0,현재생활형편CSI,서울,소비자동향,NaN,93.0,95,94.0,91.0,92.0,93.0,89.0,85.0,93.0,94.0,92.0,91.0,91.0,96.0,95.0,96.0,95.0,95.0,91.0,92.0,94.0,92.0,93.0,93.0,NaN
1,현재경기판단CSI,서울,소비자동향,NaN,87.0,86,86.0,84.0,84.0,81.0,78.0,68.0,64.0,66.0,62.0,62.0,66.0,72.0,71.0,75.0,70.0,70.0,68.0,64.0,68.0,72.0,74.0,72.0,NaN


In [ ]:
df_csi.drop(['단위','Unnamed: 28'], axis=1, inplace=True)

In [ ]:
df_csi_seoul = df_csi.mean().tolist()

### 2-5-2. 서울 이외의 지역

In [ ]:
df_others.head(2)

,CSI_CODE별,BRANCH_CODE별,2018. 01,2018. 02,2018. 03,2018. 04,2018. 05,2018. 06,2018. 07,2018. 08,2018. 09,2018. 10,2018. 11,2018. 12,2019. 01,2019. 02,2019. 03,2019. 04,2019. 05,2019. 06,2019. 07,2019. 08,2019. 09,2019. 10,2019. 11,2019. 12
0,현재생활형편CSI,부산,90.0,84.0,88.0,89.0,91.0,87.0,86.0,87.0,85.0,84.0,82.0,82.0,82.0,87.0,84.0,87.0,87.0,85.0,85.0,86.0,85.0,89.0,86.0,85.0
1,현재생활형편CSI,대구경북,89.0,91.0,91.0,92.0,92.0,89.0,87.0,87.0,90.0,88.0,87.0,87.0,86.0,88.0,87.0,89.0,89.0,88.0,88.0,83.0,85.0,85.0,87.0,87.0


In [ ]:
df_others_csi = df_others[df_others['CSI_CODE별'].apply(lambda x: x in ["현재생활형편CSI","생활형편전망CSI","현재경기판단CSI","향후경기전망CSI","소비지출전망CSI","가계수입전망CSI"])].reset_index(drop=True)

In [ ]:
df_group = df_others_csi.groupby(['BRANCH_CODE별']).mean().reset_index()

### 2-5-3. 데이터 병합

In [ ]:
df_csi_seoul = ["서울"] + df_csi_seoul

In [ ]:
df_final = df_group.append(pd.DataFrame([df_csi_seoul], columns = df_group.columns)).reset_index(drop=True)

In [ ]:
df_final.rename(columns={"BRANCH_CODE별":"시도"},inplace=True)

In [ ]:
print(df_final.shape)

(14, 25)


In [ ]:
df_final['시도'].unique()

array(['강릉', '강원', '경기', '경남', '광주전남', '대구경북', '대전충남', '부산', '울산', '인천',
       '전북', '제주', '충북', '서울'], dtype=object)

In [ ]:
df_final.to_csv('최종데이터/소비자심리지수.csv',encoding='CP949',index=False)

### 2-5-4. 날씨지수

In [ ]:
df_final = pd.read_csv('최종데이터/소비자심리지수.csv',encoding='CP949')

In [ ]:
df_final.head()

,시도,2018. 01,2018. 02,2018. 03,2018. 04,2018. 05,2018. 06,2018. 07,2018. 08,2018. 09,2018. 10,2018. 11,2018. 12,2019. 01,2019. 02,2019. 03,2019. 04,2019. 05,2019. 06,2019. 07,2019. 08,2019. 09,2019. 10,2019. 11,2019. 12,2020. 01,2020. 02,2020. 03,2020. 04,2020. 05,2020. 06,2020. 07,2020. 08,2020. 09,2020. 10,2020. 11,2020. 12
0,강릉,102.500000,98.333333,101.333333,97.333333,101.833333,99.000000,97.666667,93.000000,86.166667,86.333333,82.166667,82.666667,86.666667,88.000000,87.500000,87.000000,84.166667,86.333333,86.166667,83.166667,87.333333,86.833333,88.500000,87.666667,89.666667,86.666667,70.000000,67.166667,75.000000,79.000000,79.000000,80.000000,71.333333,78.833333,83.833333,78.333333
1,강원,103.166667,100.000000,101.666667,98.333333,101.833333,98.500000,95.000000,91.500000,86.000000,87.333333,84.500000,84.500000,87.000000,88.500000,88.166667,89.833333,86.833333,87.166667,87.000000,84.666667,88.166667,88.166667,88.333333,88.833333,90.666667,88.000000,72.833333,70.000000,75.666667,78.166667,79.166667,80.666667,72.833333,81.166667,85.000000,78.833333
2,경기,102.333333,102.666667,101.333333,100.500000,102.166667,98.833333,95.500000,91.833333,90.666667,90.500000,87.000000,87.833333,89.500000,90.666667,90.000000,92.333333,89.833333,89.500000,88.500000,86.333333,89.333333,91.166667,93.666667,93.500000,95.500000,89.666667,75.666667,71.666667,76.000000,78.500000,81.833333,83.833333,75.833333,86.500000,92.166667,84.500000
3,경남,94.666667,92.833333,93.166667,93.000000,94.666667,91.500000,88.666667,87.500000,83.666667,84.166667,80.666667,81.333333,83.000000,83.833333,84.833333,87.500000,85.666667,84.000000,84.000000,81.500000,83.500000,85.000000,87.833333,87.500000,88.666667,85.333333,71.500000,63.833333,71.333333,72.500000,76.666667,79.666667,70.500000,79.500000,85.833333,80.500000
4,광주전남,104.000000,103.666667,102.000000,102.000000,104.000000,101.500000,97.166667,93.500000,92.166667,94.333333,92.833333,91.500000,93.166667,95.500000,94.166667,94.333333,92.833333,94.000000,91.833333,88.000000,93.333333,93.833333,95.500000,96.166667,97.166667,91.000000,76.500000,73.500000,76.833333,80.166667,77.833333,82.500000,74.500000,84.666667,89.833333,83.500000


In [ ]:
df_melt = pd.melt(df_final, id_vars=['시도'])
df_melt['yymm'] = df_melt['variable'].apply(lambda x: x[:4] + x[6:8])

In [ ]:
df_melt.head()

,시도,variable,value,yymm
0,강릉,2018. 01,102.500000,201801
1,강원,2018. 01,103.166667,201801
2,경기,2018. 01,102.333333,201801
3,경남,2018. 01,94.666667,201801
4,광주전남,2018. 01,104.000000,201801


In [ ]:
def score(data, col_name):
    # 데이터 전처리
    data['시도'] = data['aws_id'].map({108:'서울', 159:'부산', 143:'대구', 112:'인천', 156:'광주',
                                     133:'대전', 152:'울산', 119:'경기', 105:'강원', 131:'충북', 232:'충남',
                                     146:'전북', 168:'전남', 136:'경북', 155:'경남', 184:'제주'})
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['yymm'] = data['year'].map(str) + data['month'].apply(lambda x: str(x) if x >= 10 else '0'+str(x))

    # score 계산
    data['score'] = np.nan
    for i in range(len(data)):
        for j in range(len(df_melt)):
            if (data.loc[i,'yymm'] == df_melt.loc[j,'yymm']) & (data.loc[i,'시도'] in df_melt.loc[j,'시도']):
                data.loc[i,'score'] = data.loc[i,col_name]*(df_melt.loc[j,'value']/100)
    return pd.DataFrame(data.groupby(['date'])['score'].mean()).reset_index()

In [ ]:
weather1 = pd.read_csv('최종데이터/weather1819.csv')
weather2 = pd.read_csv('최종데이터/weather1819_2.csv', encoding='CP949')
weather3 = pd.read_csv('최종데이터/weather_final.csv',encoding='CP949')
customer = pd.read_csv('최종데이터/소비자심리지수.csv',encoding='CP949')
online = pd.read_csv('최종데이터/data.csv',encoding='CP949')

In [ ]:
# 데이터 재구조화
df_melt = pd.melt(customer, id_vars=['시도'])
# yymm 변수 생성
df_melt['yymm'] = df_melt['variable'].apply(lambda x: x[:4] + x[6:8])
# 열 이름 변경
weather1.rename(columns={'stn_id':'aws_id'}, inplace=True)
# 필요없는 열 제거
weather2.drop(['Unnamed: 0'], axis=1, inplace=True)
# 날짜 변수
online['date'] = pd.to_datetime(online['date'], format='%Y%m%d')

#### 소비자 심리지수 반영한 날씨지수 

In [ ]:
def score(data, col_name):
    # 데이터 전처리
    data['시도'] = data['aws_id'].map({108:'서울', 159:'부산', 143:'대구', 112:'인천', 156:'광주',
                                     133:'대전', 152:'울산', 119:'경기', 105:'강원', 131:'충북', 232:'충남',
                                     146:'전북', 168:'전남', 136:'경북', 155:'경남', 184:'제주'})
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['yymm'] = data['year'].map(str) + data['month'].apply(lambda x: str(x) if x >= 10 else '0'+str(x))

    # score 계산
    data['score'] = np.nan
    for i in tqdm(range(len(data))):
        for j in range(len(df_melt)):
            if (data.loc[i,'yymm'] == df_melt.loc[j,'yymm']) & (data.loc[i,'시도'] in df_melt.loc[j,'시도']):
                data.loc[i,'score'] = data.loc[i,col_name]*(df_melt.loc[j,'value']/100)
    return pd.DataFrame(data.groupby(['date'])['score'].mean()).reset_index()

In [ ]:
# weather1에 대한 날씨지수
avg_ta = score(weather1, 'avg_ta')
max_ta = score(weather1, 'max_ta')
min_ta = score(weather1, 'min_ta')
rn_day = score(weather1, 'rn_day')
rn_hr1 = score(weather1, 'rn_hr1')
avg_ws = score(weather1, 'avg_ws')

# 열 이름 변경
avg_ta.columns = ['date', 'avg_ta']
max_ta.columns = ['date', 'max_ta']
min_ta.columns = ['date', 'min_ta']
rn_day.columns = ['date', 'rn_day']
rn_hr1.columns = ['date', 'rn_hr1']
avg_ws.columns = ['date', 'avg_ws']

In [ ]:
# weather2에 대한 날씨지수
PM10 = score(weather2, 'PM10')
PM25 = score(weather2, 'PM25')
hm_max = score(weather2, 'hm_max')
sum_ss_hr = score(weather2, 'sum_ss_hr')
max_pa = score(weather2, 'max_pa')

# 열 이름 변경
avg_ta.columns = ['date', 'avg_ta']
max_ta.columns = ['date', 'max_ta']
min_ta.columns = ['date', 'min_ta']
rn_day.columns = ['date', 'rn_day']
rn_hr1.columns = ['date', 'rn_hr1']
avg_ws.columns = ['date', 'avg_ws']

In [ ]:
# 여러 데이터 프레임 merge
dfs2 = [PM10, PM25, hm_max, sum_ss_hr, max_pa]
df_merge2 = reduce(lambda left, right: pd.merge(left, right, on='date'), dfs2)

In [ ]:
df_merge2.head(2)

,date,PM10,PM25,hm_max,sum_ss_hr,max_pa
0,2018-01-01,44.015611,21.477208,61.923313,52.276365,1020.592385
1,2018-01-02,59.179487,33.836826,74.169781,37.996677,1021.524802


#### 온라인 구매건수 merge

In [ ]:
online.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt
0,2018-01-01,F,20,식품,가공란,37,0.480964
1,2018-01-01,F,30,식품,가공란,16,0.480964


In [ ]:
merge_1 = pd.merge(online, df_merge, how='left', on='date')
df_final = pd.merge(merge_1, df_merge2, how='left', on='date')

In [ ]:
df_final.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa
0,2018-01-01,F,20,식품,가공란,37,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
1,2018-01-01,F,30,식품,가공란,16,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385


In [ ]:
df_final.to_csv('최종데이터/data_with_weather.csv', index=False, encoding='CP949')

## 2-6. 인구 가중치 `population_weight.csv`


In [ ]:
population = pd.read_csv('외부데이터/시도별 주민등록 인구현황.csv', encoding='CP949')

In [ ]:
population.head(2)

,시·도별(1),2018,2018.1,2018.2,2018.3,2018.4,2018.5,2018.6,2018.7,2018.8,2018.9,2019,2019.1,2019.2,2019.3,2019.4,2019.5,2019.6,2019.7,2019.8,2019.9
0,시·도별(1),총인구 (명),남 (명),여 (명),한국인_세대 (세대),한국인_계 (명),한국인_남 (명),한국인_여 (명),외국인_계 (명),외국인_남 (명),외국인_여 (명),총인구 (명),남 (명),여 (명),한국인_세대 (세대),한국인_계 (명),한국인_남 (명),한국인_여 (명),외국인_계 (명),외국인_남 (명),외국인_여 (명)
1,전국,53079513,26585715,26493798,22042947,51826059,25866129,25959930,1253454,719586,533868,53128391,26595111,26533280,22481466,51849861,25864816,25985045,1278530,730295,548235


In [ ]:
df_pop = population.loc[:,['시·도별(1)','2018','2019']]
df_population = df_pop.drop(0,axis=0).reset_index(drop=True)

In [ ]:
df_population['2018'] = df_population['2018'].astype('float')
df_population['2019'] = df_population['2019'].astype('float')

In [ ]:
df_population.head(2)

,시·도별(1),2018,2019
0,전국,53079513.0,53128391.0
1,서울,10049607.0,10010983.0


In [ ]:
df_weight = pd.DataFrame(columns=['시도','가중치'])

In [ ]:
df_population[df_population['시·도별(1)'].isin(['서울','경기','인천'])].loc[:,['2018','2019']]

,2018,2019
1,10049607.0,10010983.0
4,3022511.0,3029285.0
9,13485679.0,13653984.0


In [ ]:
weight = []
weight.append(df_population[df_population['시·도별(1)'].isin(['서울','경기','인천'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['강원'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['경북','대구'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['경남','부산','울산'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['전남','광주'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['전북'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['충남','대전','세종'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['충북'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['제주'])].loc[:,['2018','2019']].sum().mean())

In [ ]:
sido = ['수도권','강원','경북','경남','전남','전북','충남','충북','제주']

In [ ]:
df_weight['시도'] = sido
df_weight['가중치'] = weight

In [ ]:
df_weight['최종_가중치'] = df_weight['가중치']/df_weight['가중치'].sum()

In [ ]:
df_weight.sort_values(by='최종_가중치',ascending=False)

,시도,가중치,최종_가중치,final_가중치
0,수도권,26626024.5,0.501394,0.39
3,경남,8095822.0,0.152452,0.12
2,경북,5207889.5,0.098070,0.11
6,충남,4028170.0,0.075854,0.06
4,전남,3390919.5,0.063854,0.07
5,전북,1860085.0,0.035027,0.08
7,충북,1639755.0,0.030878,0.07
1,강원,1560942.0,0.029394,0.07
8,제주,694344.5,0.013075,0.03


In [ ]:
df_weight['final_가중치'] = round(df_weight['최종_가중치'],2)

In [ ]:
df_final = df_weight[['시도','final_가중치']]

In [ ]:
df_final.to_csv('최종데이터/population_weight.csv', encoding='CP949', index=False)

# 3. 최종 데이터

In [ ]:
# Data Load 
weather = pd.read_csv('최종데이터/weather_final2.csv', encoding='cp949')
weatherindex = pd.read_csv('최종데이터/data_with_weather.csv', encoding='cp949')
trend = pd.read_csv('외부데이터/trend_with_weather.csv', encoding='cp949')

weather.shape, weatherindex.shape, trend.shape

((11680, 29), (2056899, 18), (2568668, 12), (192, 9), (191, 9))

In [ ]:
weather = weather.rename(columns = {"날짜" : "date"})

tqdm.pandas()
weather["date"] = weather["date"].progress_apply(lambda x : pd.to_datetime(str(x), format="%Y-%m-%d"))
weatherindex["date"] = weatherindex["date"].progress_apply(lambda x : pd.to_datetime(str(x), format="%Y-%m-%d"))
trend["date"] = trend["date"].progress_apply(lambda x : pd.to_datetime(str(x), format="%Y-%m-%d"))

100%|██████████| 2568668/2568668 [03:02<00:00, 14074.06it/s]


In [ ]:
weather.head()

,date,지점번호,평균기온,최고기온,최저기온,일별강수량,1시간최대강수량,평균풍속,지역,PM10,PM25,1시간최대습도,일조시간합,최고현지기압,연,월,일,분기,요일,공휴일명,공휴일여부,주말여부,계절,체감온도,열지수,폭염여부,강수여부,year,month
0,2018-01-01,105,1.300000,5.700000,-2.100000,0.000000,0.000000,3.700000,강릉,20.066667,13.400000,25.400000,57.900000,1023.000000,2018,1,1,1,0,1월1일,1,0,3,7.502056,-39.701524,0,0,2018,1
1,2018-01-01,112,-0.300000,2.700000,-2.700000,0.000000,0.000000,1.600000,인천,37.518681,18.641758,67.200000,53.800000,1020.300000,2018,1,1,1,0,1월1일,1,0,3,9.992379,-42.924922,0,0,2018,1
2,2018-01-01,119,-1.700000,4.700000,-6.900000,0.000000,0.000000,1.000000,수원,42.782895,21.375000,84.700000,52.700000,1025.100000,2018,1,1,1,0,1월1일,1,0,3,10.136402,-45.791893,0,0,2018,1
3,2018-01-01,136,-1.000000,4.700000,-6.500000,0.000000,0.000000,2.200000,안동,39.750000,28.583333,56.400000,58.300000,1010.600000,2018,1,1,1,0,1월1일,1,0,3,8.356692,-44.375704,0,0,2018,1
4,2018-01-01,152,2.100000,6.200000,-0.400000,0.000000,0.000000,3.300000,울산,38.548193,15.433735,42.800000,58.300000,1017.100000,2018,1,1,1,0,1월1일,1,0,3,8.861429,-38.064334,0,0,2018,1


## 3-1. Data Merge 

In [ ]:
weather_cnt = weatherindex[["date", "sex", "age", "big_cat", "sm_cat", "qty", "cnt"]]

In [ ]:
# 날씨 지수 데이터 
weatherindex2 = weatherindex[['date', 'avg_ta', 'max_ta', 'min_ta', 'rn_day', 'rn_hr1', 'avg_ws', 'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa']]
weatherindex2 = weatherindex2.drop_duplicates('date', keep='first').reset_index(drop=True)

print(weatherindex2.shape)
weatherindex2.head()

(730, 12)


,date,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa
0,2018-01-01,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
1,2018-01-02,1.208896,6.014385,-3.135167,0.000000,0.000000,1.903604,59.179487,33.836826,74.169781,37.996677,1021.524802
2,2018-01-03,-0.798604,3.236083,-3.619615,0.000000,0.000000,2.623562,32.757902,18.242391,59.310396,53.216885,1023.277500
3,2018-01-04,-1.075625,2.345708,-4.636646,0.000000,0.000000,1.842729,38.685406,23.045234,63.961750,33.707167,1021.539615
4,2018-01-05,0.584646,5.053948,-2.273062,0.316688,0.087646,2.280937,46.151746,30.785155,71.747365,40.403240,1014.525396


In [ ]:
# 날짜 변수
weather2 = weatherindex2.merge(weather[['date', '연', '월', '분기', '요일', '공휴일여부', '주말여부', '계절']], on="date", how='left').drop_duplicates().reset_index(drop=True)

# 날짜 더미변수 전처리 
weather2 = pd.get_dummies(weather2, columns=['연'], prefix='연', drop_first=True) 
weather2 = pd.get_dummies(weather2, columns=['월'], prefix='월', drop_first=True) 
weather2 = pd.get_dummies(weather2, columns=['분기'], prefix='분기', drop_first=True) 
weather2 = pd.get_dummies(weather2, columns=['요일'], prefix='요일', drop_first=True) 
weather2 = pd.get_dummies(weather2, columns=['계절'], prefix='계절', drop_first=False) 

print(weather2.shape)
weather2.head()

(730, 39)


,date,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,계절_0,계절_1,계절_2,계절_3
0,2018-01-01,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2018-01-02,1.208896,6.014385,-3.135167,0.000000,0.000000,1.903604,59.179487,33.836826,74.169781,37.996677,1021.524802,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,2018-01-03,-0.798604,3.236083,-3.619615,0.000000,0.000000,2.623562,32.757902,18.242391,59.310396,53.216885,1023.277500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,2018-01-04,-1.075625,2.345708,-4.636646,0.000000,0.000000,1.842729,38.685406,23.045234,63.961750,33.707167,1021.539615,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,2018-01-05,0.584646,5.053948,-2.273062,0.316688,0.087646,2.280937,46.151746,30.785155,71.747365,40.403240,1014.525396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [ ]:
weather2 = weather2.rename(columns = {"계절_0" : "봄", "계절_1" : "여름", "계절_2" : "가을", "계절_3" : "겨울"})

In [ ]:
# 검색량 데이터 + 날씨 지수 데이터 
final = trend.merge(weather2, on='date', how='left')
final = final.merge(weather_cnt, on=["date", "sex", "age", "big_cat", "sm_cat", "qty"], how='left')

print(final.shape)
final.head()

(2568668, 51)


,date,sex,age,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,봄,여름,가을,겨울,cnt
0,2018-01-01,F,20,식품,가공란,37.000000,0.000000,10.693870,4.091370,0.000000,0.000000,0.044400,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.480964
1,2018-01-01,F,30,식품,가공란,16.000000,0.000000,12.429690,6.352940,0.000000,0.512820,0.424170,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.480964
2,2018-01-01,F,40,식품,가공란,9.000000,0.000000,13.022490,4.880370,0.000000,0.406500,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.480964
3,2018-01-01,F,50,식품,가공란,3.000000,0.000000,11.690800,4.533500,1.219510,0.909090,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.480964
4,2018-01-01,M,20,식품,가공란,13.000000,0.000000,11.230320,3.612960,0.389100,2.222220,0.040600,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.480964


In [ ]:
# 공통 더미변수 전처리 
final["sex"] = final["sex"].progress_apply(lambda x : 1 if x=='F' else 0)
final = pd.get_dummies(final, columns=['age'], prefix='age', drop_first=False)

100%|██████████| 2568668/2568668 [00:02<00:00, 1116496.12it/s]


## 3-2. `final_0622.csv`  
* 범주형 변수가 더미변수인 데이터셋

In [ ]:
# 최종 dataframe 
final.to_csv(os.path.join("최종데이터", "final_0622.csv"), index=False, encoding="cp949")

In [ ]:
# 범주형 
weather3 = weatherindex2.merge(weather[['date', '연', '월', '분기', '요일', '공휴일여부', '주말여부', '계절']], on="date", how='left').drop_duplicates().reset_index(drop=True)

# 검색량 데이터 + 날씨 지수 데이터 
final2 = trend.merge(weather3, on='date', how='left')
final2 = final2.merge(weather_cnt, on=["date", "sex", "age", "big_cat", "sm_cat", "qty"], how='left')

print(final2.shape)
final2.head()

(2568668, 31)


,date,sex,age,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,연,월,분기,요일,공휴일여부,주말여부,계절,cnt
0,2018-01-01,F,20,식품,가공란,37.000000,0.000000,10.693870,4.091370,0.000000,0.000000,0.044400,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,2018,1,1,0,1,0,3,0.480964
1,2018-01-01,F,30,식품,가공란,16.000000,0.000000,12.429690,6.352940,0.000000,0.512820,0.424170,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,2018,1,1,0,1,0,3,0.480964
2,2018-01-01,F,40,식품,가공란,9.000000,0.000000,13.022490,4.880370,0.000000,0.406500,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,2018,1,1,0,1,0,3,0.480964
3,2018-01-01,F,50,식품,가공란,3.000000,0.000000,11.690800,4.533500,1.219510,0.909090,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,2018,1,1,0,1,0,3,0.480964
4,2018-01-01,M,20,식품,가공란,13.000000,0.000000,11.230320,3.612960,0.389100,2.222220,0.040600,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,2018,1,1,0,1,0,3,0.480964


## 3-3. `final_categorical_0622.csv` 
* 범주형 변수 변형 전 데이터셋

In [ ]:
final2.to_csv(os.path.join("최종데이터", "final_categorical_0622.csv"), index=False, encoding="cp949")